# import Libraries

In [ ]:
!sudo apt install poppler-utils

In [ ]:
!sudo apt-get install build-essential

In [ ]:
!sudo apt install -y gconf-service libasound2 libatk1.0-0 libc6 libcairo2 libcups2 libdbus-1-3 libexpat1 libfontconfig1 libgcc1 libgconf-2-4 libgdk-pixbuf2.0-0 libglib2.0-0 libgtk-3-0 libnspr4 libpango-1.0-0 libpangocairo-1.0-0 libstdc++6 libx11-6 libx11-xcb1 libxcb1 libxcomposite1 libxcursor1 libxdamage1 libxext6 libxfixes3 libxi6 libxrandr2 libxrender1 libxss1 libxtst6 ca-certificates fonts-liberation libappindicator1 libnss3 lsb-release xdg-utils wget libgbm-dev

In [ ]:
!pip install pyppeteer
import asyncio
from pyppeteer import launch
import pandas as pd
import datetime

# Test for library

In [ ]:
import asyncio
import nest_asyncio
from pyppeteer import launch

nest_asyncio.apply()

async def main():
    browser = await launch({
        'headless': True,
        'args': [
            '--no-sandbox',
            '--disable-setuid-sandbox',
            '--single-process',
            '--no-zygote',
        ],
    })
    page = await browser.newPage()
    await page.goto('http://example.com')
    await page.screenshot({'path': 'example.png'})
    await browser.close()

asyncio.get_event_loop().run_until_complete(main())

[INFO] Starting Chromium download.
INFO:pyppeteer.chromium_downloader:Starting Chromium download.
100%|██████████| 109M/109M [00:00<00:00, 125Mb/s]
[INFO] Beginning extraction
INFO:pyppeteer.chromium_downloader:Beginning extraction
[INFO] Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/588429
INFO:pyppeteer.chromium_downloader:Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/588429


# Get urls for all courses

Here, we have brought the titles of all the free courses on the Coursera platform, and they numbered a thousand courses.

In [ ]:
async def extract_ratings(numberOfPage):
    browser = await launch({
        'headless': True,
        'args': [
            '--no-sandbox',
            '--disable-setuid-sandbox',
            '--single-process',
            '--no-zygote',
        ],
    })
    page = await browser.newPage()
    url = f'https://www.coursera.org/courses?query=free&page={numberOfPage}'
    await page.goto(url)

    # Assuming you are selecting elements using a CSS selector
    rating_elements = await page.querySelectorAll('.css-11uuo4b')  # Update with the correct CSS selector

    # Collect all ratings
    ratings = []
    for element in rating_elements:
        rating_handle = await element.getProperty('textContent')
        rating = await rating_handle.jsonValue()
        ratings.append(rating)

    # Assuming you are selecting elements using a CSS selector
    elements = await page.querySelectorAll('.cds-CommonCard-titleLink')  # Update with the correct CSS selector

    # Collect all URLs
    urls = []
    for element in elements:
        url_handle = await element.getProperty('href')
        url = await url_handle.jsonValue()
        urls.append(url)

    await browser.close()
    return ratings,urls



In [ ]:
# Threshold for filtering ratings
filtered_urls = []
for numberOfPage in range(1,85):
    ratings,urls = asyncio.get_event_loop().run_until_complete(extract_ratings(numberOfPage))
    # Create a new list to store URLs with ratings below the threshold
    filtered_urls += urls#[url for rating, url in zip(ratings, urls) if float(rating) < threshold]

In [ ]:
filtered_urls[:10]

['https://www.coursera.org/specializations/gpu-programming',
 'https://www.coursera.org/specializations/human-resource-management',
 'https://www.coursera.org/learn/protect-and-grow-your-business',
 'https://www.coursera.org/learn/instructional-design-foundations-applications',
 'https://www.coursera.org/specializations/machine-learning-trading',
 'https://www.coursera.org/learn/finance-fundamentals',
 'https://www.coursera.org/specializations/software-architecture-big-data',
 'https://www.coursera.org/specializations/mlops-machine-learning-duke',
 'https://www.coursera.org/learn/rust-fundamentals',
 'https://www.coursera.org/specializations/advanced-leadership-skills',
 'https://www.coursera.org/specializations/advanced-embedded-linux-development',
 'https://www.coursera.org/specializations/deep-learning-healthcare',
 'https://www.coursera.org/professional-certificates/google-it-support-korean',
 'https://www.coursera.org/specializations/machine-learning-reinforcement-finance']

In [ ]:
len(filtered_urls)

1000

saved that urls in txt file

In [ ]:
file_path = '/content/drive/MyDrive/ASC/all_courses.txt'

# Writing the URLs to the text file
with open(file_path, 'w') as file:
    for url in filtered_urls:
        file.write(url + '\n')

# get reviews from all courses

After that, we collected all the reviews for each course, and their numbers varied between the courses, and in the end we obtained approximately 478,775 reviews.

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_all_review_texts(title):
    review_texts = []
    for numberOfPage in range(1, 150):
        url = f'{title}/reviews?page={numberOfPage}'
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            review_elements = soup.select('.reviewText')

            if review_elements:
                review_texts.extend([element.get_text(strip=True) for element in review_elements])
            else:
                break
        else:
            print(f"Failed to fetch data for {url}. Status code: {response.status_code}")

    return review_texts

def get_reviews_from_all_courses(index):
    file_path = '/content/drive/MyDrive/ASC/all_courses.txt'
    start_line = index - 1000
    end_line = index
    all_review_texts = []

    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            if start_line <= i < end_line:
                print(i)
                all_review_texts.extend(get_all_review_texts(line.strip()))
            elif i >= end_line:
                break

    if all_review_texts:
        for i, review_text in enumerate(all_review_texts, start=1):
            print(f"Review {i} Text:", review_text)
    else:
        print("No review texts found.")

    return all_review_texts


In [ ]:
all_review_texts = []

In [ ]:
temp = get_reviews_from_all_courses(1000)
all_review_texts.extend(temp)

In [ ]:
df = pd.DataFrame({'Review Text': all_review_texts})
df.to_csv('/content/drive/MyDrive/ASC/0-1000 Courses.csv', index=False)